In [10]:
import pandas as pd
import numpy as np
 
from tqdm import tqdm

In [11]:
rfm_score = pd.read_csv('../rawdata/result.csv')

In [12]:
rfm_score.head()

,CustomerID,Recency,Frequency,Monetary,Recency_R,Frequency_F,Monetary_M
0,12346.0,2011-01-18 10:01:00,1,77183.60,1,2,5
1,12747.0,2011-12-07 14:34:00,9,3489.74,5,5,5
2,12748.0,2011-12-09 12:20:00,175,29491.60,5,5,5
3,12749.0,2011-12-06 09:56:00,5,4090.88,5,4,5
4,12820.0,2011-12-06 15:12:00,4,942.34,5,4,4


In [13]:
rfm_score = rfm_score[['CustomerID', 'Monetary', 'Recency_R', 'Frequency_F', 'Monetary_M']]

In [14]:
rfm_score.head()

,CustomerID,Monetary,Recency_R,Frequency_F,Monetary_M
0,12346.0,77183.60,1,2,5
1,12747.0,3489.74,5,5,5
2,12748.0,29491.60,5,5,5
3,12749.0,4090.88,5,4,5
4,12820.0,942.34,5,4,4


In [15]:
def get_score(level, data, reverse = False):
    '''
    Description :
    level안에 있는 원소를 기준으로
    1 ~ len(level)+ 1 까지 점수를 부여하는 함수
    
    Parameters :
    level = 튜플 또는 리스트 타입의 숫자형 데이터이며 반드시 오름차순으로 정렬되어 있어야함.
    예 - [1,2,3,4,5] O, [5,4,3,2,1] X, [1,3,2,10,4] X 
    data = 점수를 부여할 데이터. 순회가능한(iterable) 데이터 형식
    reverse = 점수가 높을 때 그에 해당하는 값을 낮게 설정하고 싶을 때 True
    return :
    점수를 담고 있는 리스트 반환
    '''
    score = [] 
    for j in range(len(data)): 
        for i in range(len(level)): 
            if data[j] <= level[i]: 
                score.append(i+1) 
                break 
            elif data[j] > max(level): 
                score.append(len(level)+1) 
                break 
            else: 
                continue
    if reverse:
        return [len(level)+2-x for x in score]
    else:
        return score 
 
grid_number = 100 ## 눈금 개수, 너무 크게 잡으면 메모리 문제가 발생할 수 있음.
weights = []
for j in range(grid_number+1):
    weights += [(i/grid_number,j/grid_number,(grid_number-i-j)/grid_number)
                  for i in range(grid_number+1-j)]
num_class = 5 ## 클래스 개수
class_level = np.linspace(1,5,num_class+1)[1:-1] ## 클래스를 나누는 지점을 정한다.
total_amount_of_sales = rfm_score['Monetary'].sum() ## 구매금액 총합 = 총 매출

In [16]:
max_std = 0 ## 표준편차 초기값
for w in tqdm(weights,position=0,desc = '[Finding Optimal weights]'):
    ## 주어진 가중치에 따른 고객별 점수 계산
    score = w[0]*rfm_score['Recency_R'] + \
                        w[1]*rfm_score['Frequency_F'] + \
                        w[2]*rfm_score['Monetary_M'] 
    rfm_score['Class'] = get_score(class_level,score,True) ## 점수를 이용하여 고객별 등급 부여
    ## 등급별로 구매금액을 집계한다.
    grouped_rfm_score = rfm_score.groupby('Class')['Monetary'].sum().reset_index()
    
    ## 제약조건 추가 - 등급이 높은 고객들의 매출이 낮은 등급의 고객들보다 커야한다.
    grouped_rfm_score = grouped_rfm_score.sort_values('Class')
    
    temp_monetary = list(grouped_rfm_score['Monetary'])
    if temp_monetary != sorted(temp_monetary,reverse=True):
        continue
        
    ## 클래스별 구매금액을 총구매금액으로 나누어 클래스별 매출 기여도 계산
    grouped_rfm_score['Monetary'] = grouped_rfm_score['Monetary'].map(lambda x : x/total_amount_of_sales)
    std_sales = grouped_rfm_score['Monetary'].std() ## 매출 기여도의 표준편차 계산
    if max_std <= std_sales:
        max_std = std_sales ## 표준편차 최대값 업데이트
        optimal_weights = w  ## 가중치 업데이트

[Finding Optimal weights]: 100%|██████████| 5151/5151 [03:47<00:00, 22.63it/s]


In [17]:
print(optimal_weights)

(0.0, 0.26, 0.74)


### 가중치와 RFM 점수를 이용하여 등급 부여

In [18]:
score = optimal_weights[0]*rfm_score['Recency_R'] + \
        optimal_weights[1]*rfm_score['Frequency_F'] + \
        optimal_weights[2]*rfm_score['Monetary_M'] ## 고객별 점수 계산
 
rfm_score['Class'] = get_score(class_level,score,True) ## 고객별 등급 부여

### 각 등급별 매출 기여도 확인

In [19]:
## 클래스별 고객 수 계산
temp_rfm_score1 = rfm_score.groupby('Class')['CustomerID'].count().reset_index().rename(columns={'CustomerID':'Count'})
 
## 클래스별 구매금액(매출)계산
temp_rfm_score2 = rfm_score.groupby('Class')['Monetary'].sum().reset_index()
 
## 클래스별 매출 기여도 계산
temp_rfm_score2['Monetary'] = temp_rfm_score2['Monetary'].map(lambda x : x/total_amount_of_sales)
 
## 데이터 결합
result_df = pd.merge(temp_rfm_score1,temp_rfm_score2,how='left',on=('Class'))

In [20]:
result_df

,Class,Count,Monetary
0,1,913,0.759984
1,2,629,0.115659
2,3,748,0.070348
3,4,762,0.037681
4,5,762,0.016328
